In [1]:
import pickle

- xgb : 0.72
- cb : 0.71
- deep : 0.65

In [3]:
reply_to_idx_dict = dict()
with open("../data/test_replies.txt", "r") as f:
    for line in f:
        idx, reply = line.strip().split("\t")
        reply_to_idx_dict[reply] = idx

In [9]:
with open("../data/test_replies.txt", "r") as f:
    test_replies = [line.strip().split("\t")[1] for line in f]

In [4]:
reply_to_idx_dict

{'개빡치게 하네 진짜': '199',
 '결혼 했어?': '198',
 '고양이 너무 좋아': '197',
 '괜찮아': '196',
 '교회가기 귀찮아': '195',
 '그 사람도 나를 좋아할까?': '194',
 '그런식으로 말 하지 마': '193',
 '그만 좀 해': '192',
 '길가다가 넘어졌어': '191',
 '김상무 진짜 또 지랄이야': '190',
 '나 결혼은 할 수 있을까?': '189',
 '나 내일 잘 할 수 있겠지?': '188',
 '나 대신 걔 좀 혼내 줘': '187',
 '나 대신 일해줘': '186',
 '나 대신 회사 가주면 안돼?': '185',
 '나 더위 먹은 거 같아': '184',
 '나 바다 보고 싶어': '183',
 '나 배고파': '182',
 '나 살찐거 같아?': '181',
 '나 아파': '180',
 '나 엄청 인기 많아': '179',
 '나 없는동안 뭐하고 있었어?': '178',
 '나 오늘 꿈꿨어': '177',
 '나 오늘 누가 예쁘다고 해줬다~~~': '176',
 '나 오늘 번호 따였다~~~': '175',
 '나 위로 좀 해줘': '174',
 '나 이번 달에 왜 이렇게 많이 썼지': '173',
 '나 이제 점심 먹을라고': '172',
 '나 이제 퇴근한다': '171',
 '나 잘거야': '170',
 '나 정도면 잘 생겼지': '169',
 '나 졸려': '168',
 '나 진짜 화났어': '167',
 '나는 결혼 안하고싶어': '166',
 '나는 왜 이 모양일까?': '165',
 '나는 잘하는게 없는거같아': '164',
 '나랑 결혼해 줄래?': '163',
 '나랑 놀아줘': '162',
 '나랑 얘기 좀 해': '161',
 '나를 좋아하는 사람은 아무도 없어': '160',
 '난 너만 믿는다': '159',
 '난 오늘도 야근이야': '158',
 '남자친구 있어?': '157',
 '내가 또 술먹으면 개다': '156',
 '내가 해도 저것보단 잘 치겠

In [5]:
xgb_result = pickle.load(open("../dump/xgb_result.pkl", "rb"))
cb_result = pickle.load(open("../dump/cb_result.pkl", "rb"))
deep_result = pickle.load(open("../dump/deep_result.pkl", "rb"))

In [7]:
with open("../data/test_queries.txt", "r") as f:
    test_queries = [line.strip().split("\t")[1] for line in f]

In [10]:
from tqdm import tqdm

In [12]:
predictions = list()

In [13]:
for query in tqdm(test_queries):
    xgb_dict = {r: score for r, score in xgb_result[query]}
    cb_dict = {r: score for r, score in cb_result[query]}
    deep_dict = {r: score for r, score in deep_result[query]}
    total_dict = dict()
    for reply in test_replies:
        total_dict[reply] = xgb_dict[reply]*0.72 + cb_dict[reply]*0.71 + deep_dict[reply]*0.65
    predictions.append([query, sorted(total_dict.items(), key=lambda x: x[1], reverse=True)[0][0]])

100%|██████████| 600/600 [00:00<00:00, 1584.07it/s]


In [15]:
submission = list()
for p in predictions:
    reply = p[1]
    submission.append(reply_to_idx_dict[reply])

In [17]:
import pandas as pd

In [18]:
s = pd.read_csv("../submission/sample_submission.csv")

In [20]:
s["id_script"] = pd.Series(submission)

In [22]:
s.to_csv("../submission/submission_1.csv", index=False)

In [24]:
xgb_predictions = list()
for query in tqdm(test_queries):
    xgb_predictions.append([query, sorted(xgb_result[query], key=lambda x: x[1], reverse=True)[0][0]])

100%|██████████| 600/600 [00:00<00:00, 13754.52it/s]


In [26]:
s = pd.read_csv("../submission/sample_submission.csv")
s["id_script"] = pd.Series([reply_to_idx_dict[r] for r in [p[1] for p in xgb_predictions]])

In [28]:
s.to_csv("../submission/submission_xgb.csv", index=False)